## Build a Conversational AI Chatbot with OpenAI and Gradio

*[Coding along with the Udemy online course [LLM Engineering: Master AI & Large Language Models](https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models/) by Ed Donner; GitHub repo can be found at [github.com/ed-donner/llm_engineering](https://github.com/ed-donner/llm_engineering)]*

In [1]:
from openai import OpenAI
import anthropic
import google.generativeai
from IPython.display import Markdown, display, update_display
import pandas as pd
from pprint import pprint
import requests
from bs4 import BeautifulSoup
from typing import List
# and finally, let's import gradio as a new feature here
# https://www.gradio.app/guides/quickstart
import gradio as gr

In [2]:
openai_api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

anthropic_api_key = pd.read_csv("~/tmp/anthropic/anthropic-key-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

google_api_key = pd.read_csv("~/tmp/google-gemini/gemini-key-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

# connect to openai
openai = OpenAI(api_key=openai_api_key)

# connect to anthropic
claude = anthropic.Anthropic(api_key=anthropic_api_key)

# connect to gemini
google.generativeai.configure(api_key=google_api_key)

Don't be a fool and sent your api key to github
Don't be a fool and sent your api key to github
Don't be a fool and sent your api key to github


In [3]:
# OPENAI MODEL
MODEL = "gpt-4o-mini"

In [5]:
system_message = "You're a helpful assistant"

__As a reminder:__ For a simple conversation between user and system prompts are organized into lists like this:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

This structure can also be used to reflect a longer conversation history with different roles:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

The advantage of this approach is that we can use it to engage in a longer interaction with history. 

We will write a function `chat(message, history)` where:
**message** is the prompt to use
**history** is a list of pairs of user message with assistant's reply

```
[
    ["user said this", "assistant replied"],
    ["then user said this", "and assistant replied again],
    ...
]
```

In the next step we will convert this history into the prompt style for OpenAI, then call the OpenAI API.

#### __An AI Assistant is a very common Gen AI use case__

LLM based Chatbots are remarkably effective at conversation:

- Friendly, knowledgable __persona__
- Ability to maintain __context__ between messages
- Subject matter __expertise__ to answer questions

#### __The Use of Prompts with our Assistant__

__The System Prompt:__

- Set tone
- Establish ground-rules, like "If you don't know the answer, just say no."
- Provide critical background context

__Context:__

- During the conversation, insert context to give more relevant background information pertaining to the topic

__Multi-Shot Prompting:__

- Provide example conversations to prime for specific scenarios, train on conversational style and demonstrate complex interactions


<img src="../../assets/images/prompting-assistants-1.png" with="70%" align="left" />

*Screenshot take from the Udemy online course [LLM Engineering: Master AI & Large Language Models](https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models/) by Ed Donner; go and watch it, it's definitely worth the time*

We will write a function `chat(message, history)` where:
**message** is the prompt to use
**history** is a list of pairs (a list with sub-lists) of user message with assistant's reply

```
[
    ["user said this", "assistant replied"],
    ["then user said this", "and assistant replied again],
    ...
]
```

The `chat`function we're going to write is a __a particular type of function that Gradio expects with it's chat interfaces__. The function will return the next response to this chat. We need to iterate row by row through this structure and the build the following structure out of it:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```


In [6]:
# converting history into the prompt style for OpenAI
# we write a particular type of function that Gradio expects
def chat(message, history):
    # (1) setting up list of messages and populating it with the system prompt at the very start
    # system_message was defined above
    messages = [{"role": "system", "content": system_message}]

    # (2) iterating through history
    for user_message, assistant_message in history:
        # the we append the user message and the assistant message each time
        # each row from history turns into two roles in this list
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role": "user", "content": message})

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    # calling openai with the messages
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

### Adding Gradio into the Mix

In [7]:
# turn function into user interface that has an instant message style interaction
# ChatInterface expects a single function
gr.ChatInterface(fn=chat).launch()

/Users/juergenkober/Library/Caches/pypoetry/virtualenvs/llm-engineering-E34XR2a1-py3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': "You're a helpful assistant"}, {'role': 'user', 'content': 'Hello there"'}]
History is:
[['Hello there"', 'Hello! How can I assist you today?']]
And messages is:
[{'role': 'system', 'content': "You're a helpful assistant"}, {'role': 'user', 'content': 'Hello there"'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'I want to buy a tie'}]
History is:
[['Hello there"', 'Hello! How can I assist you today?'], ['I want to buy a tie', 'Great! Here are some things to consider when buying a tie:\n\n1. **Occasion**: Is it for a formal event, a business setting, or casual wear? This will influence the style and material you choose.\n\n2. **Material**: Common materials include silk (for a luxurious look), cotton (more casual), or polyester (affordable and easy to maintain).\n\n3. **Color and Pattern**: Solid colors are versatile, while patterns like stripes, florals, or polka dots c

### Improving the Persona with the system message

In [9]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."
system_message

"You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales evemt.'Encourage the customer to buy hats if they are unsure what to get."

In [10]:
# chat function once again, this time without print statements
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    # adding in the list the latest message the user is sending
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [11]:
gr.ChatInterface(fn=chat).launch()

/Users/juergenkober/Library/Caches/pypoetry/virtualenvs/llm-engineering-E34XR2a1-py3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [12]:
# adding another line to the system message
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [13]:
system_message

"You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales evemt.'Encourage the customer to buy hats if they are unsure what to get.\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!"

In [14]:
gr.ChatInterface(fn=chat).launch()

/Users/juergenkober/Library/Caches/pypoetry/virtualenvs/llm-engineering-E34XR2a1-py3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [15]:
# one more hacky adjustment to the system message
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})

    # this is very hacky, don't try this at home kids!
    if 'belt' in message:
        messages.append({"role": "system", "content": "For added context, the store does not sell belts, \
but be sure to point out other items on sale"})
    
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [16]:
gr.ChatInterface(fn=chat).launch()

/Users/juergenkober/Library/Caches/pypoetry/virtualenvs/llm-engineering-E34XR2a1-py3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
